In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [2]:
def create_soup(url):
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='cp949')
    return soup

[arcid, press, title, content, journalist, date, link, category]

# 테스트

## 뉴스 리스트 스크래핑

In [103]:
date = '20230130'
category = 'society'
page = 1
url = f'https://news.daum.net/breakingnews/{category}?page={page}&regDate={date}'

soup = create_soup(url)

In [104]:
news_list = soup.find_all('strong', attrs={'class':'tit_thumb'})

In [333]:
soup = create_soup('https://news.daum.net/breakingnews/society?page=170&regDate=20230129')
paging = soup.find('span', attrs={'class':'inner_paging'}).find_all(text=True)
paging = [i.strip() for i in paging if i != '\n']
paging[-1]

'다음'

In [334]:
url = 'https://news.daum.net/breakingnews/society?page=178&regDate=20230129'
soup = create_soup(url)
paging = soup.find('span', attrs={'class':'inner_paging'}).find_all(text=True)
paging = [i.strip() for i in paging if i != '\n']
paging

['이전', '171', '172', '173', '174', '175', '176', '177', '현재 페이지', '178']

In [345]:
paging[-1]

'178'

In [105]:
articles = []
for i in tqdm(range(len(news_list)-4)):
    article = []
    title = news_list[i].find('a').text
    link = news_list[i].find('a')['href']
    date = news_list[i].find('a')['href'].split('/')[-1][:8]
    arcid = news_list[i].find('a')['href'].split('/')[-1][8:]
    press = news_list[i].find('span').text.split('·')[0].strip()
    category = url.split('/')[-1].split('?')[0]
    article = [arcid, press, title, date, link, category]
    articles.append(article)

100%|███████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 14563.56it/s]


In [106]:
pd.DataFrame(articles, columns=['arcid', 'press', 'title', 'date', 'link', 'category'])

,arcid,press,title,date,link,category
0,215114044,KBS,강기정 시장-시민단체 간담회…사안마다 격돌,20230130,https://v.daum.net/v/20230130215114044,society
1,215110043,KBS,3년 만에 실내 마스크 해제…“아직은 쓰는 게 편해요”,20230130,https://v.daum.net/v/20230130215110043,society
2,215050037,KBS,‘밀렵’ 여전…매년 ‘덫·올무’ 7천여 개 적발,20230130,https://v.daum.net/v/20230130215050037,society
3,215025034,KBS,“쓰레기 늘었는데” 소각장 증설 반대…지자체 골머리,20230130,https://v.daum.net/v/20230130215025034,society
4,215024033,머니투데이,"파키스탄 이슬람사원서 자폭 테러…""180여명 사상""",20230130,https://v.daum.net/v/20230130215024033,society
5,215002028,한국경제,"""귀신 내쫓아야""…친딸 폭행해 숨지게 한 무속인 아버지",20230130,https://v.daum.net/v/20230130215002028,society
6,214938027,KBS,유치원·어린이집 하나로…“2025년 시행 목표”,20230130,https://v.daum.net/v/20230130214938027,society
7,214921025,KBS,[일하다 죽지 않게] 중대재해법 기소 11건 불과…법 개정만 검토?,20230130,https://v.daum.net/v/20230130214921025,society
8,214918024,KBS,"교통문화지수, 원주 4년 연속 1위…속초·인제·고성도 ‘A’",20230130,https://v.daum.net/v/20230130214918024,society
9,214915023,KBS,전국혁신도시협 “2차 공공기관 혁신도시 우선 배치”,20230130,https://v.daum.net/v/20230130214915023,society


## 링크로 본문 스크래핑

In [299]:
link = 'https://v.daum.net/v/20230129093900580'
article_soup = create_soup(link)
article = article_soup.find_all('section')[1]
article = article.find_all(True, attrs={'dmcf-ptype':'general'})
article = [i.text.strip() for i in article]

In [300]:
article

['오늘(29일) 0시 기준 코로나19 신규 환자는 18,871명으로 집계됐습니다.',
 '1주일 전보다 2천여 명 늘었고 2주 전보다는 만3천여 명 줄었습니다.',
 '신규 환자 가운데 국내 발생은 만 8,834명, 해외 유입은 37명입니다.',
 '위중증 환자는 420명으로 12일 연속 4백 명대를 나타냈습니다.',
 '사망자는 29명으로 누적 사망자는 3만 3,390명입니다.',
 'YTN 김현아 (kimhaha@ytn.co.kr)',
 "※ '당신의 제보가 뉴스가 됩니다'",
 '[카카오톡] YTN 검색해 채널 추가',
 '[전화] 02-398-8585',
 '[메일] social@ytn.co.kr',
 '[저작권자(c) YTN 무단전재 및 재배포 금지]']

In [304]:
len('사망자는 29명으로 누적 사망자는 3만 3,390명입니다.')

32

In [305]:
if len(article) == 1:
    n = 1
elif len(article) == 0 or len(article[0]) < 20:
    print('pass')
for i in range(len(article)-1, -1, -1):
    if len(article[i]) > 30:
        n = i+1
        break
content = [i.strip() for i in article[:n] if i != '']
content = ' '.join(content)
journalist = article_soup.select('div>span')[0].text.split('기자')[0].strip() + ' 기자'

In [291]:
journalist

'원성윤 기자'

In [292]:
content

"[아이뉴스24 원성윤 기자] /원성윤 기자(better2017@inews24.com) ▶네이버 채널에서 '아이뉴스24'를 구독해주세요. ▶재밌는 아이뉴스TV 영상보기▶아이뉴스24 바로가기 [ⓒ 아이뉴스24 무단전재 및 재배포 금지]"

# 함수 정리

In [9]:
def content_scraper(link):
    article_soup = create_soup(link)
    article = article_soup.find_all('section')[1]
    article = article.find_all(True, attrs={'dmcf-ptype':'general'})
    article = [i.text.strip() for i in article]
    if len(article) == 1:
        n = 1
    elif len(article) == 0 or len(article[0]) < 20:
        return False
    for i in range(len(article)-1, -1, -1):
        if len(article[i]) > 30:
            n = i+1
            break
    content = [i.strip() for i in article[:n] if i != '']
    content = ' '.join(content)
    journalist = article_soup.select('div>span')[0].text.split('기자')[0].strip() + ' 기자'
    return content, journalist

In [10]:
def get_articles(date, category):
    articles = []
    for page in range(1, 1000):
        url = f'https://news.daum.net/breakingnews/{category}?page={page}&regDate={date}'
        soup = create_soup(url)
        
        # 마지막 리스트 체크
        print(f'check page: {page} to', end=' ')
        paging = soup.find('span', attrs={'class':'inner_paging'}).find_all(text=True)
        paging = [i.strip() for i in paging if i != '\n']
        print(f'{paging[-1]}\t\t\t', end='\r')
        if paging[-1] == str(page):
            end = page
            break

    for page in tqdm(range(1, end+1)):
        url = f'https://news.daum.net/breakingnews/{category}?page={page}&regDate={date}'
        soup = create_soup(url)
        
        news_list = soup.find_all('strong', attrs={'class':'tit_thumb'})
        for i in range(len(news_list)-4):
            article = []
            title = news_list[i].find('a').text
            link = news_list[i].find('a')['href']
            # print(link, end='\r')
            
            date = news_list[i].find('a')['href'].split('/')[-1][:8]
            arcid = news_list[i].find('a')['href'].split('/')[-1][8:]
            press = news_list[i].find('span').text.split('·')[0].strip()
            if press in ban_list:
                continue
            category = url.split('/')[-1].split('?')[0]
            checker = content_scraper(link)
            if not checker:
                continue
            content, journalist = checker
            
            article = [arcid, press, title, content, journalist, date, link, category]
            articles.append(article)
    df = pd.DataFrame(articles, columns=['arcid', 'press', 'title', 'content', 'journalist', 'date', 'link', 'category'])
    return df

In [5]:
ban_list = ['코리아중앙데일리']

In [6]:
data = get_articles('20230129', 'society')

100%|████████████████████████████████████████████████████████████████████████████████| 178/178 [06:16<00:00,  2.11s/it]


In [7]:
data

,arcid,press,title,content,journalist,date,link,category
0,235946000,아시아경제,130만 명 불러 모은 '화천산천어축제' 29일 폐막,[아시아경제 라영철 기자] 3년 만에 재개한 '2023 화천산천어축제'가 흥행에 성...,라영철 기자,20230129,https://v.daum.net/v/20230129235946000,society
1,235701991,노컷뉴스,"2023 화천산천어축제 폐막…""대표 겨울축제 건재 재확인""","핵심요약\n\n 폭우 대비 대응, 신속한 제설 등 한발 앞선 ...",강원CBS 진유정 기자,20230129,https://v.daum.net/v/20230129235701991,society
2,235502989,한겨레,1월30일 궂긴소식,"◆ 조규용씨 별세: 배월선씨 남편, 세현(사진작가, 서울 중구문화재단 사장) 종현 ...",한겨레 기자,20230129,https://v.daum.net/v/20230129235502989,society
3,235400986,SBS,"부산 온천교사거리서 차량 연쇄 추돌…""신호위반 추정""",오늘(29일) 저녁 6시쯤 부산 동래구 온천동 온천교사거리에서 60대 A 씨가 몰던...,김덕현 기자,20230129,https://v.daum.net/v/20230129235400986,society
4,235311983,조선비즈,쇼트트랙 안현수 국내복귀 무산… 성남시청 코치직 2배수에서 제외,러시아로 귀화해 러시아 쇼트트랙 대표팀 선수와 중국 쇼트트랙 대표팀 코치 등으로 활...,최효정 기자,20230129,https://v.daum.net/v/20230129235311983,society
...,...,...,...,...,...,...,...,...
2376,000039805,아이뉴스24,김은숙 작가가 쏘아올린 '학교 폭력'의 짙은 그늘 [원성윤의 人어바웃],"[아이뉴스24 원성윤 기자] ""내 소원이 뭐였는 줄 아니? 나도 언젠가는 너의 이름...",원성윤 기자,20230129,https://v.daum.net/v/20230129000039805,society
2377,000036803,아이뉴스24,[주말엔 뷰티] PX화장품 1위 달팽이 크림 왜 없어질까,[아이뉴스24 홍수현 기자] 군마트(PX) 내 부동의 인기 상품 1위를 달리던 '달...,홍수현 기자,20230129,https://v.daum.net/v/20230129000036803,society
2378,000034801,더팩트,'우여곡절' 마주 앉는 오세훈-전장연…반전은 불투명,"2일 면담하지만 의제 두고 '동상이몽'시 ""탈시설 의견수렴"", 전장연 ""이동권·사망...",이헌일 기자,20230129,https://v.daum.net/v/20230129000034801,society
2379,000031800,더팩트,"[폴리스스토리]""믿어줘서 고마워요""…성범죄 피해자의 '최후 보루'",경기북부청 경기북동부해바라기센터 장윤정 수사팀장처벌 가능 여부 떠나 '듣는 사람' ...,최의종 기자,20230129,https://v.daum.net/v/20230129000031800,society


In [13]:
data.to_csv('20230129_society.csv')

# 기간 설정 스크래핑

In [256]:
from datetime import date
from dateutil.rrule import rrule, DAILY

In [257]:
start_date = date(2023, 1, 29)
end_date = date(2023, 1, 29)

In [264]:
data = pd.DataFrame()
for date in tqdm(list(rrule(DAILY, dtstart=start_date, until=end_date))):
    # print(date.strftime('%Y%m%d'))
    data = pd.concat([data, get_articles(date.strftime('%Y%m%d'), 'society')])
data.reset_index(drop=True, inplace=True)

  0%|                                                                                            | 0/1 [01:28<?, ?it/s]


IndexError: list index out of range